# Anthropic Claude

In the v0.2.30 release of AutoGen we support Anthropic Client.

Claude is a family of large language models developed by Anthropic and designed to revolutionize the way you interact with AI. Claude excels at a wide variety of tasks involving language, reasoning, analysis, coding, and more. The models are highly capable, easy to use, and can be customized to suit your needs.

In this notebook, we demonstrate how to use Anthropic Claude model for AgentChat in AutoGen.

## Features

Additionally, this client class provides support for function/tool calling and will track token usage and cost correctly as per Anthropic's API costs (as of June 2024).

## Requirements
To use Anthropic Claude with AutoGen, first you need to install the `pyautogen["anthropic]` package.

To try out the function call feature of Claude model, you need to install `anthropic>=0.23.1`.


In [ ]:
# !pip install pyautogen
!pip install pyautogen["anthropic"]

## Set the config for the Anthropic API

You can add any parameters that are needed for the custom model loading in the same configuration list.

It is important to add the `api_type` field and set it to a string that corresponds to the client type used: `anthropic`.

Example:
```
[
    {
        "model": "claude-3-sonnet-20240229",
        "api_key": "your Anthropic API Key goes here",
        "api_type": "anthropic",
        "temperature": 0.5,
        "top_p": 0.2, # Note: It is recommended to set temperature or top_p but not both.
        "max_tokens": 10000,
    },
    {
        "model":"claude-3-opus-20240229",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.0",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-2.1",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
    {
        "model":"claude-3.0-haiku",
        "api_key":"your api key",
        "api_type":"anthropic",
    },
]
```

### Alternative

As an alternative to the api_key key and value in the config, you can set the environment variable `ANTHROPIC_API_KEY` to your Anthropic API key.

Linux/Mac:
```
export ANTHROPIC_API_KEY="your Anthropic API key here"
```
Windows:
```
set ANTHROPIC_API_KEY=your_anthropic_api_key_here
```

In [1]:
import os

from typing_extensions import Annotated

import autogen

config_list_claude = [
    {
        # Choose your model name.
        "model": "claude-3-5-sonnet-20240620",
        # You need to provide your API key here.
        "api_key": os.getenv("ANTHROPIC_API_KEY"),
        "api_type": "anthropic",
    }
]

# Coding Example with Two Agent

## Construct Agents

Construct a simple conversation between a User proxy and an ConversableAgent based on Claude-3 model.


In [2]:
assistant = autogen.AssistantAgent(
    "assistant",
    llm_config={
        "config_list": config_list_claude,
    },
)

user_proxy = autogen.UserProxyAgent(
    "user_proxy",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": "coding",
        "use_docker": False,
    },
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    max_consecutive_auto_reply=1,
)

## Initiate Chat

In [3]:
user_proxy.initiate_chat(
    assistant, message="Write a python program to print the first 10 numbers of the Fibonacci sequence."
)

user_proxy (to assistant):

Write a python program to print the first 10 numbers of the Fibonacci sequence.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Certainly! I'll write a Python program to print the first 10 numbers of the Fibonacci sequence. Here's the code:

```python
# filename: fibonacci.py

def fibonacci(n):
    fib_sequence = [0, 1]
    
    while len(fib_sequence) < n:
        next_num = fib_sequence[-1] + fib_sequence[-2]
        fib_sequence.append(next_num)
    
    return fib_sequence

# Print the first 10 numbers of the Fibonacci sequence
n = 10
result = fibonacci(n)
print(f"The first {n} numbers of the Fibonacci sequence are:")
print(result)
```

This Python script does the following:

1. We define a function called `fibonacci` that takes an argument `n`, which is the number of Fibonacci numbers we want to generate.
2. Inside the function, we initialize the sequence with the first two numbers: [0, 1].
3

/Users/qingyunwu/Documents/github/autogen/autogen/oai/anthropic.py:272: UserWarning: Cost calculation not available for model claude-3-5-sonnet-20240620
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


assistant (to user_proxy):

Great! The code executed successfully, and we have the correct output. Let's verify the result:

The first 10 numbers of the Fibonacci sequence are indeed:
[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

This is correct because:
1. The sequence starts with 0 and 1.
2. Each subsequent number is the sum of the two preceding numbers:
   - 1 = 0 + 1
   - 2 = 1 + 1
   - 3 = 1 + 2
   - 5 = 2 + 3
   - 8 = 3 + 5
   - 13 = 5 + 8
   - 21 = 8 + 13
   - 34 = 13 + 21

The program has successfully generated and printed the first 10 numbers of the Fibonacci sequence. The task is complete, and the result is verified to be correct.

TERMINATE

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Write a python program to print the first 10 numbers of the Fibonacci sequence.', 'role': 'assistant'}, {'content': 'Certainly! I\'ll write a Python program to print the first 10 numbers of the Fibonacci sequence. Here\'s the code:\n\n```python\n# filename: fibonacci.py\n\ndef fibonacci(n):\n    fib_sequence = [0, 1]\n    \n    while len(fib_sequence) < n:\n        next_num = fib_sequence[-1] + fib_sequence[-2]\n        fib_sequence.append(next_num)\n    \n    return fib_sequence\n\n# Print the first 10 numbers of the Fibonacci sequence\nn = 10\nresult = fibonacci(n)\nprint(f"The first {n} numbers of the Fibonacci sequence are:")\nprint(result)\n```\n\nThis Python script does the following:\n\n1. We define a function called `fibonacci` that takes an argument `n`, which is the number of Fibonacci numbers we want to generate.\n2. Inside the function, we initialize the sequence with the first two numbers: [0, 1].\n3. We use a while loop 

# Function Call in Latest Anthropic API 
Anthropic just announced that tool use is now supported in the Anthropic API. To use this feature, please install `anthropic>=0.23.1`.

## Register the function

In [4]:
@user_proxy.register_for_execution()  # Decorator factory for registering a function to be executed by an agent
@assistant.register_for_llm(
    name="get_weather", description="Get the current weather in a given location."
)  # Decorator factory for registering a function to be used by an agent
def preprocess(location: Annotated[str, "The city and state, e.g. Toronto, ON."]) -> str:
    return "Absolutely cloudy and rainy"

In [5]:
user_proxy.initiate_chat(
    assistant,
    message="What's the weather in Toronto?",
)

user_proxy (to assistant):

What's the weather in Toronto?

--------------------------------------------------------------------------------
assistant (to user_proxy):

***** Suggested function call: get_weather *****
Arguments: 
{"location": "Toronto, ON"}
************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_weather...


[runtime logging] log_function_use: autogen logger is None


user_proxy (to assistant):

***** Response from calling function (get_weather) *****
Absolutely cloudy and rainy
********************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

Based on the result from the `get_weather` function, I can provide you with the current weather information for Toronto:

The weather in Toronto is currently absolutely cloudy and rainy.

This means that:
1. The sky is completely covered with clouds, with no visible clear patches.
2. It is currently raining in Toronto.

If you're in Toronto or planning to go there, you should be prepared for wet conditions. It would be advisable to:
- Carry an umbrella
- Wear waterproof clothing or a raincoat
- Use appropriate footwear for wet conditions
- Be cautious while driving, as roads may be slippery

Keep in mind that weather can change, so if you need more specific or updated information (like temperature, wind speed, 

ChatResult(chat_id=None, chat_history=[{'content': "What's the weather in Toronto?", 'role': 'assistant'}, {'function_call': {'arguments': '{"location": "Toronto, ON"}', 'name': 'get_weather'}, 'content': None, 'role': 'assistant'}, {'content': 'Absolutely cloudy and rainy', 'name': 'get_weather', 'role': 'function'}, {'content': "Based on the result from the `get_weather` function, I can provide you with the current weather information for Toronto:\n\nThe weather in Toronto is currently absolutely cloudy and rainy.\n\nThis means that:\n1. The sky is completely covered with clouds, with no visible clear patches.\n2. It is currently raining in Toronto.\n\nIf you're in Toronto or planning to go there, you should be prepared for wet conditions. It would be advisable to:\n- Carry an umbrella\n- Wear waterproof clothing or a raincoat\n- Use appropriate footwear for wet conditions\n- Be cautious while driving, as roads may be slippery\n\nKeep in mind that weather can change, so if you need m